In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [2]:
# 데이터 출처 - http://data.nsdi.go.kr/dataset/20180927ds0059 (국가공간정보포털-행정경계(읍면동))
# shp 파일 경로
shp_file_path = "전국자연환경조사_5차_조류_2019_전국/전국자연환경조사_5차_조류_2019_전국.shp"

# shp 파일을 GeoDataFrame으로 읽기
gdf = gpd.read_file(shp_file_path, encoding='cp949')
gdf

,id,학명,한글보통명,위도,경도,조사일자,조사연도,시도명,개체수,geometry
0,0080202100012247,Emberiza elegans,노랑턱멧새,37.729167,126.729167,2019-03-01,2019,경기도,3,POINT (176124.801 569973.678)
1,0080202100044713,Microscelis amaurotis,직박구리,35.437500,128.437500,2019-03-01,2019,경상남도,2,POINT (330528.978 316582.661)
2,0080202100012251,Streptopelia orientalis,멧비둘기,37.729167,126.729167,2019-03-01,2019,경기도,3,POINT (176124.801 569973.678)
3,0080202100013014,Anser albifrons,쇠기러기,37.812500,126.729167,2019-03-01,2019,경기도,110,POINT (176151.581 579223.050)
4,0080202100012254,Anas poecilorhyncha,흰뺨검둥오리,37.729167,126.729167,2019-03-01,2019,경기도,2,POINT (176124.801 569973.678)
...,...,...,...,...,...,...,...,...,...,...
88646,0080202100018426,Upupa epops,후투티,35.562500,128.437500,2019-12-01,2019,경상남도,1,POINT (330326.856 330452.815)
88647,0080202100018430,Pica pica,까치,35.562500,128.437500,2019-12-01,2019,경상남도,2,POINT (330326.856 330452.815)
88648,0080202100018427,Buteo buteo,말똥가리,35.562500,128.437500,2019-12-01,2019,경상남도,1,POINT (330326.856 330452.815)
88649,0080202100018428,Emberiza elegans,노랑턱멧새,35.562500,128.437500,2019-12-01,2019,경상남도,12,POINT (330326.856 330452.815)


In [9]:
len(gdf['taxon_f_nm'].unique())

123

In [7]:
gdf['taxon_nm'].unique()

array(['개개비', '멧비둘기', '중대백로', '촉새', '호랑지빠귀', '휘파람새', '쇠붉은뺨멧새', '흰물떼새',
       '참새', '검은댕기해오라기', '매사촌', '유리딱새', '개똥지빠귀', '제비', '흰날개해오라기', '검은딱새',
       '후투티', '뻐꾸기', '해오라기', '바다제비', '직박구리', '어치', '청딱다구리', '괭이갈매기',
       '왜가리', '덤불해오라기', '말똥가리', '물까치', '오색딱다구리', '흰눈썹지빠귀', '붉은목지빠귀', '황로',
       '쇠박새', '소쩍새', '멧도요', '흰뺨검둥오리', '때까치', '노랑턱멧새', '딱새', '박새', '곤줄박이',
       '꿩', '바다쇠오리', '붉은머리오목눈이', '울새', '힝둥새', '꼬까참새', '흰배멧새', '동박새',
       '흰등밭종다리', '긴발톱멧새', '멧종다리', '진홍가슴', '되새', '쇠솔새', '흰배지빠귀', '솔새사촌',
       '까치', '큰소쩍새', '쇠백로', '쇠오리', '알락오리', '물총새', '쇠부엉이', '쥐발귀개개비',
       '검은가슴물떼새', '황조롱이', '쑥새', '산솔새', '북방검은머리쑥새', '오목눈이', '큰기러기1',
       '제주휘파람새', '종다리', '꼬마물떼새', '알락쇠오리', '귀제비', '굴뚝새', '진박새', '밭종다리',
       '멧새', '재갈매기', '노랑눈썹솔새', '바다직박구리', '검은지빠귀', '중부리도요', '되지빠귀', '넓적부리',
       '쇠물닭', '노랑딱새', '콩새', '흰제비갈매기', '쇠유리새', '검은머리방울새', '검은슴새', '북방개개비',
       '붉은가슴밭종다리', '붉은허리개개비', '숲새', '붉은배지빠귀', '제비딱새', '큰유리새', '황금새',
       '노랑눈썹멧새', '쇠솔딱새', '쇠찌르레기', '흰눈썹황금새', '노랑때까치', '노랑지빠귀', '알락꼬리쥐발귀'

In [ ]:
pd.read_excel("")

In [3]:
# 데이터 출처 - http://data.nsdi.go.kr/dataset/20180927ds0059 (국가공간정보포털-행정경계(읍면동))
# shp 파일 경로
shp_file_path = "전국자연환경조사_4차_조류_2014_2018_전국/전국자연환경조사_4차_조류_2014_2018_전국.shp"

# shp 파일을 GeoDataFrame으로 읽기
gdf = gpd.read_file(shp_file_path, encoding='cp949')
gdf

,id,학명,한글보통명,위도,경도,조사일자,조사연도,시도명,개체수,geometry
0,008000000002277521,Turdus hortulorum,되지빠귀,37.816517,127.572653,2014-04-01,2014,강원도,None,POINT (250422.80140 579788.84590)
1,008000000002277506,Dendrocopos leucotos,큰오색딱다구리,37.764022,127.581431,2014-04-02,2014,강원도,None,POINT (251231.95480 573966.99150)
2,008000000002285773,Saxicola torquatus,검은딱새,37.860611,127.784139,2014-04-11,2014,강원도,None,POINT (269003.54320 584818.31820)
3,008000000002283728,Picus canus,청딱다구리,37.828750,127.758111,2014-04-11,2014,강원도,None,POINT (266741.79560 581262.94280)
4,008000000002283678,Dendrocopos major,오색딱다구리,37.725803,127.774025,2014-05-16,2014,강원도,None,POINT (268237.39680 569847.85510)
...,...,...,...,...,...,...,...,...,...,...
258174,008000000002175492,Passer montanus,참새,90.000000,127.000000,2018-10-16,2018,None,26,POINT (1844674401947755.25000 1844674393761355...
258175,008000000002272760,Microscelis amaurotis,직박구리,90.000000,127.000000,2018-10-16,2018,None,27,POINT (1844674401947755.25000 1844674393761355...
258176,008000000002099801,Passer montanus,참새,90.000000,127.000000,2018-10-16,2018,None,22,POINT (1844674401947755.25000 1844674393761355...
258177,008000000002252271,Microscelis amaurotis,직박구리,90.000000,127.000000,2018-10-16,2018,None,3,POINT (1844674401947755.25000 1844674393761355...
